In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install streamlit-webrtc


     |████████████████████████████████| 8.2MB 9.7MB/s 
     |████████████████████████████████| 163kB 60.1MB/s 
     |████████████████████████████████| 112kB 54.8MB/s 
     |████████████████████████████████| 4.6MB 56.7MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 122kB 63.2MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=c414323a52d0765e395d1ee3895771271f4099e4b14f5dbe7fa32a8b04557cd8
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


     |████████████████████████████████| 747kB 8.3MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=17eb85cca37171f4b8bbf8f9db72a03220d70fb909542db257d5bc50e551c77f
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok
     |████████████████████████████████| 798kB 9.9MB/s 
     |████████████████████████████████| 2.0MB 21.1MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 37.2MB 80kB/s 
     |████████████████████████████████| 3.2MB 51.9MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 245kB 55.2MB/s 
  Created wheel for netifaces: filename=netifaces-0.10.9-cp37-cp37m-linux_x86_64.whl size=37425 sha256=d3ae4eb5d9d8118e9fc26b2186a54f3e514e9c021f46c85aa0b350cbca99e15c
  Stored in directory: /root/.cache/pip/wheels/23/8f/f3/7054578f04c904f70757c5c85a6e2823baa69d42365526e

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-04-10 19:53:51--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.209.152.48, 54.226.184.31, 54.159.163.191, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.209.152.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.06M  18.2MB/s    in 0.8s    

2021-04-10 19:53:52 (18.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14746350/14746350]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!./ngrok authtoken 1qweyuAJ0cKxjhGAHdWsGqfWFls_2x1bwHu9papYMQmBTrM5A

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
%%writefile app.py
import numpy as np
import pandas as pd
import tensorflow as tf
import streamlit as st
from datetime import date
from streamlit_webrtc import VideoTransformerBase, webrtc_streamer
import os
import dlib
import cv2 as cv
import tempfile
from skimage import io
import matplotlib.pyplot as plt
import csv
from csv import writer
from csv import reader
import math
from PIL import Image, ImageOps
from skimage import io
import matplotlib.pyplot as plt


def img_to_encoding(image_path, model):
    img1 = cv.imread(image_path, 1)
    img = img1[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

def predict_score(img_path, EmoNet, show=False):

    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(48, 48))
    img_tensor = tf.keras.preprocessing.image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)         
    img_tensor /= 255.                                      

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    model_class = np.argmax(EmoNet.predict(img_tensor), axis=-1)
    if model_class == 0:
      return int(3)
    elif model_class == 1:
      return int(2)
    else:
      return int(1)

def identify(image_path, database, model):

  encoding = img_to_encoding(image_path, model)
  min_dist = 100
  for (name, db_enc) in database.items():
    dist = np.linalg.norm(encoding-db_enc)

  if dist < min_dist:
    min_dist = dist
    identity = name
  if min_dist > 0.7:
    return "Not Found"
  return identity

def getCount():
  with open('/content/sample_data/data.csv','r',newline="") as csvfile:
    writer_object = writer(csvfile)
    df=reader(csvfile)
    dg=pd.read_csv('/content/sample_data/data.csv')
    cnt=-1
    flag=False
    name_list=[]
    count=0
    for col in dg:
      count=count+1
    return count, col

def add_to_database(NAME,DATE,VALUE):
  if not os.path.exists("/content/sample_data/data.csv"):
    with open("/content/sample_data/data.csv",'w',newline='') as csvfile:
      data={'Name':[NAME],DATE:[VALUE]}
      df=pd.DataFrame(data,columns=['Name',DATE])
      df.to_csv("/content/sample_data/data.csv",index=False,header=True)
  else:
    with open("/content/sample_data/data.csv",'r',newline="")  as csvfile:
      dg=pd.read_csv("/content/sample_data/data.csv")
      df=reader(csvfile)
      flag1=False
      count1=-1
      y=0
      prevy=0
      for col in dg:
        count1=count1+1
        if col==DATE:
          flag1=True
          break
      count=-2
      flag=False
      for row in df:
        count=count+1
        if row[0]==NAME:
          flag=True
          y=row[count1]
          prevy=row[count1-1]
          break
      if flag==True:
        z=0
        if flag1==True:
          z=0

          if count1>=2:
            #z=max(int(VALUE),int(int(float(y))-int(float(prevy))))
            if y.len()==0:
              z=int(VALUE)
              z=int(z)+int(float(prevy))
            else:
              z=max(int(VALUE),int(int(float(y))-int(float(prevy))))
              z=int(z)+int(float(prevy)) 
          else:
            z=max(int(VALUE),int(float(y)))

          dg.loc[count,DATE]=int(z)
          dg.to_csv("/content/sample_data/data.csv",index=False)
        else:
          dg[DATE]=""
          z=int(VALUE)
          if count1>=1:
            z=int(z)+int(float(y))
          dg.loc[count,DATE]=int(z)
          dg.to_csv("/content/sample_data/data.csv",index=False)  
      else:
        with open("/content/sample_data/data.csv",'a+',newline="") as csvfile1:
          z=0
          if flag1==True:
            z=int(VALUE)
            if count1>=1:
              z=int(z)+int(float(prevy))
            dg.loc[count+1,'Name']=NAME
            dg.loc[count+1,DATE]=int(VALUE)
            dg.to_csv("/content/sample_data/data.csv",index=False)
          else:
            dg[DATE]=" "
            z=int(VALUE)
            dg.loc[count+1,'Name']=NAME
            dg=pd.read_csv("/content/sample_data/data.csv")
            dg.loc[count+1,DATE]=int(z)+int(float(prevy))
            dg.to_csv("/content/sample_data/data.csv",index=False)

def scan_database(DATE, threshold):
  with open("/content/sample_data/data.csv",'r',newline="") as csvfile:
    writer_object = writer(csvfile)
    df=reader(csvfile)
    dg=pd.read_csv("/content/sample_data/data.csv")
    cnt=-1
    flag=False
    name_list={}
    for col in dg:
      cnt=cnt+1
      if col==DATE:#date given by user
        break
    for row in df:
      if row[0]=="Name":
        continue
      if(int(float(row[cnt]))<threshold):
        name_list[row[0]] = int(float(row[cnt]))

  return name_list

def curr_value(DATE,NAME):
  with open("/content/sample_data/data.csv",'r',newline="") as csvfile:
    dg=pd.read_csv("/content/sample_data/data.csv")
    df=reader(csvfile)
    count=-1
    flag=False;
    for col in dg:
      count=count+1
      if col==DATE:#date given by user
           break
    x=0
    for row in df:
      if row[0]==NAME:
         x=row[count]
       #print(x)
    return x

def view_database(database):
  today = date.today()
  d1 = today.strftime("%d/%m/%Y")
  for keys in database:
    st.subheader(keys)
    # value = 24
    c, d = getCount()    
    value = curr_value(d1,keys)
    value = math.ceil(((int(float(value))*1.0)/(3*c)) * 100)
    st.write(str(value) + "%")
    progress_bar = st.progress(value)
    if value < 40:
      status = st.write("Sad")
    else:
      status = st.write("Happy")

def scan_vdo(f, date):

  tfile = tempfile.NamedTemporaryFile(delete=False) 
  tfile.write(f.read())


  vf = cv.VideoCapture(tfile.name)

  stframe = st.empty()

  while vf.isOpened():
    ret, frame = vf.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    cv.imwrite("frame.jpg", frame)
    img_path = '/content/frame.jpg'
    person = identify(tf.keras.preprocessing.image.load_img(img_path, target_size=(48, 48)), database, FrNet)
    if person!="Not Found":
      add_to_database(person,f_date,predict_score(img_path, EmoNet))

def save_uploaded_file(uploadedfile):

  img_path = os.path.join("/content",uploadedfile.name)
  with open(os.path.join("/content",uploadedfile.name),"wb") as f:
     f.write(uploadedfile.getbuffer())
  return img_path

def main():

  # class VideoTransformer(VideoTransformerBase):
  #   def transform(self, frame):
  #     img = frame.to_ndarray(format="bgr24")
  #     img = cv.cvtColor(cv.Canny(img, 100, 200), cv.COLOR_GRAY2BGR)

  #     return img

  def triplet_loss(y_true, y_pred, alpha = 0.2):    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis = -1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis = -1)
    basic_loss = pos_dist- neg_dist + alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))    
    return loss

  FrNet = tf.keras.models.load_model('/content/drive/MyDrive/Hack36/FrNet', custom_objects={'triplet_loss': triplet_loss})
  EmoNet = tf.keras.models.load_model('/content/drive/MyDrive/Hack36/emonet2.h5')
  database = {}
  phn_db = {}

  def initialize_db():

    today = date.today()
    # d1 = today.strftime("%d/%m/%Y")
    d1 = "11/4/2021"
    database["danielle"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/danielle.png", FrNet)
    add_to_database("danielle", d1, predict_score("/content/drive/MyDrive/Hack36/images/danielle.png", EmoNet))
    phn_db["danielle"] = "8976442222"
    
    database["younes"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/younes.jpg", FrNet)
    add_to_database("younes", d1, predict_score("/content/drive/MyDrive/Hack36/images/younes.jpg", EmoNet))
    phn_db["younes"] = "4354563465"

    database["tian"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/tian.jpg", FrNet)
    add_to_database('tian', d1, predict_score('/content/drive/MyDrive/Hack36/images/tian.jpg', EmoNet))
    phn_db["tian"] = "25435345324"

    database["andrew"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/andrew.jpg", FrNet)
    add_to_database("andrew", d1, predict_score("/content/drive/MyDrive/Hack36/images/andrew.jpg", EmoNet))
    phn_db["andrew"] = "3642646625"

    database["kian"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/kian.jpg", FrNet)
    add_to_database("kian", d1, predict_score("/content/drive/MyDrive/Hack36/images/kian.jpg", EmoNet))
    phn_db["kian"] = "626434625"

    database["dan"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/dan.jpg", FrNet)
    add_to_database("dan", d1, predict_score("/content/drive/MyDrive/Hack36/images/dan.jpg", EmoNet))
    phn_db["dan"] = "2636323443"

    database["sebastiano"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/sebastiano.jpg",FrNet)
    add_to_database("sebastiano", d1, predict_score("/content/drive/MyDrive/Hack36/images/sebastiano.jpg", EmoNet))
    phn_db["sebastiano"] = "7364512323"

    database["bertrand"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/bertrand.jpg", FrNet)
    add_to_database("bertrand", d1, predict_score("/content/drive/MyDrive/Hack36/images/bertrand.jpg", EmoNet))
    phn_db["bertrand"] = "342652655"

    database["kevin"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/kevin.jpg", FrNet)
    add_to_database("kevin", d1, predict_score('/content/drive/MyDrive/Hack36/images/kevin.jpg', EmoNet))
    phn_db["kevin"] = "345423534"

    database["felix"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/felix.jpg", FrNet)
    add_to_database("felix", d1, predict_score("/content/drive/MyDrive/Hack36/images/felix.jpg", EmoNet))
    phn_db["felix"] = "345134535"

    database["benoit"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/benoit.jpg", FrNet)
    add_to_database("benoit", d1, predict_score("/content/drive/MyDrive/Hack36/images/benoit.jpg", EmoNet))
    phn_db["benoit"] = "21416778"

    database["arnaud"] = img_to_encoding("/content/drive/MyDrive/Hack36/images/arnaud.jpg", FrNet)
    add_to_database('arnaud', d1, predict_score("/content/drive/MyDrive/Hack36/images/arnaud.jpg", EmoNet))
    phn_db["arnaud"] = "0988987665"

  initialize_db()
  st.title("Altruist ~ We care <3")
  menu = ["Home", "Register into Database", "View Database", "Upload Video", "Scan Database"]
  choice = st.sidebar.selectbox('Menu', menu)
  if choice == 'Home':
    st.title("Home")
  elif choice == "Register into Database":
    st.title("Register into Database")
    name = st.text_input("Your full name")
    phn = st.text_input("Phone number")
    st.write("Upload a picture")
    img = st.file_uploader("Upload a picture",  type=["jpg", "png"])
    st.write('You selected `%s`' % img)
    if img is not None:
      file_details = {"FileName":img.name,"FileType":img.type}
      img_path = save_uploaded_file(img)
      st.write(img.name)
      submit = st.button("Submit")
      if submit:
        st.image(img, width=None)
        phn_db[name] = str(phn)
        img2 = tf.keras.preprocessing.image.load_img(img_path, target_size=(96, 96))
        img2 = img2.save("gg.jpg")
        img_path2 = "/content/gg.jpg"
        database[name] = img_to_encoding(img_path2, FrNet)

  elif choice == "View Database":
    st.title("View Database")
    view_database(database)
  elif choice == "Upload Video":
    st.write("Enter date on which footage was taken")
    f_date = st.text_input("DD/MM/YY")
    vdo = st.file_uploader("Upload a Video")
    st.write('You selected `%s`' % vdo)
    if vdo is not None:
      v = vdo.read()
      st.video(v)
      video_file = open(vdo, 'rb')
      video_bytes = video_file.read()
      st.video(video_bytes)
      scan_vdo(vdo, f_date)
  elif choice == "Scan Database":
    c, d = getCount()
    t = math.ceil(0.4*3*c)
    mylist = scan_database(d, t)
    view_database(mylist)     
    submit = st.button('Click to send text (*Frontend)')
    if submit:
      message()

if __name__ == '__main__':
  main()

Writing app.py


In [ ]:
!ls

app.py	drive  ngrok  ngrok-stable-linux-amd64.zip  sample_data


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')


In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: http://46056bd04785.ngrok.io


In [5]:
!streamlit run ./app.py


  👋 Welcome to Streamlit!

  If you're one of our development partners or you're interested in getting
  personal technical support or Streamlit updates, please enter your email
  address below. Otherwise, you may leave the field blank.

  Email:  ^C
2022-04-09 20:39:24.527 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')